# Quantum Circuit Ansatze

In this notebook we will go over the Quantum Circuit Ansatze that are used in this project.
We construct 3 anzatze:

    1. Hardware Efficient Ansatz
    2. Tree Tensor Network
    3. Multiscale Entanglement Renormalization Ansatz
These ansatze are constructed as templates first and then applied to Hamiltonians later when constructing the final circuit.

In [1]:
import pennylane as qml
import numpy as np
from hea  import hea_circuit
from ttn  import ttn_circuit
from mera import mera_circuit, get_num_mera_gates

## Hardware Efficient Ansatz (HEA)
<img src="images/HEA.png" alt="Drawing" style="width: 500px;"/>

This anzatz consists of a layer of single-gate arbitrary rotations followed by a ring of CNOT gates as shown in the above figure. This is then repeated some number of times (either by a constant, logarithmically in system size, or linear in system size). 

In [2]:
num_qubits = 4
hea_depth  = 2 # how many times will we repeat the layer
num_param_sets = 3*hea_depth*num_qubits # each layer has num_qubits rotation gates, each having 3 parameters.
hea_ansatz = hea_circuit(num_qubits)

dev = qml.device('default.qubit', wires=num_qubits)
@qml.qnode(dev)
def hea(params):
    hea_ansatz(params, list(range((num_qubits))))
    return qml.expval(qml.PauliZ(0))

params = np.random.uniform(low=-np.pi / 2, high=np.pi / 2, size=(num_param_sets, 3))

Z_0 = hea(params)
print(f"<Z_0> = {Z_0}")

<Z_0> = 0.2792519540491323


## Tree Tensor Network (TTN)
<img src="images/TTN.png" alt="Drawing" style="width: 500px;"/>

This Ansatz resembles a binary tree for a quantum circuit. The number of gates is linear in system size, and so the number of parameters are linear in system size. In this ansatz, we have an option to have the unitaries in each layer have the same parameters, thus allowing us to reduce the total number of parameters to be logarithmic in system size. We can also choose how we want to specify our two-gate unitaries in constructing the circuit

In [3]:
num_qubits = 8
two_qubit_gate = qml.templates.ArbitraryUnitary

ttn_ansatz = ttn_circuit(num_qubits, two_qubit_gate, fix_layers=False)
dev = qml.device('default.qubit', wires=num_qubits)

@qml.qnode(dev)
def ttn(params):
    ttn_ansatz(params, num_qubits)
    return qml.expval(qml.PauliZ(0))

num_params_per_gate = 15 #arbitrary 2-gate unitary has 15 parameters
num_gates = num_qubits - 1
num_params = num_params_per_gate * num_gates
params = np.pi*(np.random.rand(num_params) - 1.0)

Z_0 = ttn(params)
print(f"<Z_0> = {Z_0}")

<Z_0> = 0.230126058929646


## Multiscale Entanglement Renormalization Ansatz (MERA)
<img src="images/MERA.png" alt="Drawing" style="width: 500px;"/>

The MERA is similar to a tree tensor network, with the addition of unitary "disentangelers" between branches as shown above with the blue boxes. In our construction, we can fix the unitaries in each layer to have the same parameters, and we can choose if we want our MERA to be periodic.

In [4]:
num_qubits = 8
periodic, fix_layers = True, False
mera_ansatz = mera_circuit(num_qubits, periodic=periodic, fix_layers=fix_layers)
dev = qml.device('default.qubit', wires=num_qubits)

@qml.qnode(dev)
def mera(params):
    mera_ansatz(params, num_qubits)
    return qml.expval(qml.PauliZ(0))

num_params_per_gate = 15 #arbitrary 2-gate unitary has 15 parameters
num_gates = get_num_mera_gates(num_qubits, periodic=periodic, fix_layers=fix_layers)
num_params = num_params_per_gate * num_gates
params = np.pi*(np.random.rand(num_params) - 1.0)
Z_0 = mera(params)
print(f"<Z_0> = {Z_0}")

<Z_0> = -0.1779793514532204
